In [ ]:
from sklearn import tree
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score,classification_report,f1_score

In [ ]:
attr = pd.read_csv('attr.txt', sep='\t',
                           names=['link', 'length', 'direction', 'path_class', 'speed_class', 'LaneNum', 'speed_limit',
                                  'level', 'width'], header=None)

In [ ]:
train = pd.read_csv('is_train_2019070' + str(1) + '.txt')
test = pd.read_csv('is_test.csv')
train = train.merge(attr, on='link', how='left')
test = test.merge(attr, on='link', how='left')

In [ ]:
id_col = 'link'
use_train_feats = [i for i in train.columns if i not in ['link', 'label', 'current_slice_id', 'future_slice_id', 'label_pred']]

In [ ]:
train_user_id = train[id_col].unique()

In [ ]:
X_train= train.loc[:,use_train_feats]
X_train = np.array(X_train)

In [ ]:
y = np.array(train['label'])
y = y.reshape(-1,1)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
from sklearn import tree
from sklearn.metrics import accuracy_score

class AdaBoostClassifier:
    def __init__(self, n_weakers_limit):
        self.weights = []
        self.classifiers = []
        self.n_weakers_limit = n_weakers_limit
        self.ws = []
        pass

    def is_good_enough(self,X_train,y_train):
        if((len(self.classifiers) > self.n_weakers_limit) or
                accuracy_score(y_train, self.predict(X_train))>=0.9):
            return True
        else:
            #print("准确率：" + str(accuracy_score(y_train, self.predict(X_train))))
            print(classification_report(y_train, self.predict(X_train), digits=4))
            report = f1_score(y_train, self.predict(X_train), average=None)
            print('Score: ', report)
            return False
        pass

    def fit(self,X_train,y_train):
        w = np.ones((1, len(X_train))) / len(y_train)
        flag = 0
        while (flag == 0 or (not(self.is_good_enough(X_train,y_train)))):
        #for i in range(2):
            # 定义并训练基分类器决策树
            flag = 1
            clf = tree.DecisionTreeClassifier\
            (class_weight="balanced", criterion='gini', max_depth=1, splitter="best")
            predit_tree = clf.fit(X_train, y_train, w.reshape(-1))
            #predit_tree = clf.fit(X_train, y_train)
            self.classifiers.append(predit_tree)
            # 计算错误率
            correct_rate = predit_tree.score(X_train, y_train)
            error_rate = 1 - correct_rate
            # 计算分类器权重
            alpha = 1 / 2 * np.log((1 - error_rate) / error_rate)
            # 存储分类器权重
            self.weights.append(alpha)
            # 计算预测结果
            result = predit_tree.predict(X_train).reshape(-1, 1)
            # 更新数据集权重参数
            first_part = -alpha * y_train
            second_part = np.exp(first_part * result)
            third_part = (w.reshape(-1, 1) * second_part)
            z_t = np.sum(third_part)
            w = w.reshape(-1, 1) * second_part / z_t
            self.ws.append(w)
            print(alpha)
        pass

    def predict(self, X_train, threshold=0):
        scores = []
        lenth = len(self.classifiers)
        mmm_0 = np.zeros((len(X_train),1))
        mmm_1 = np.zeros((len(X_train),1))
        mmm_2 = np.zeros((len(X_train),1))
        
        for i in range(lenth):
            kk = self.classifiers[i].predict(X_train).reshape(-1,1)
            sum_0 = np.where(kk == 0, 1, 0)
            sum_0 = sum_0.astype(np.float64)
            sum_0 *= self.weights[i]
            mmm_0 += sum_0
            #print(mmm_0)
            sum_1 = np.where(kk == 1, 1, 0)
            sum_1 = sum_1.astype(np.float64)
            sum_1 *= self.weights[i]
            mmm_1 += sum_1

            sum_2 = np.where(kk == 2, 1, 0)
            sum_2 = sum_2.astype(np.float64)
            sum_2 *= self.weights[i]
            mmm_2 += sum_2

        mmm = np.concatenate((mmm_0,mmm_1,mmm_2),axis=1)
        return np.argmax(mmm,axis=1)     
        pass


    @staticmethod
    def save(model, filename):
        with open(filename, "wb") as f:
            pickle.dump(model, f)

    @staticmethod
    def load(filename):
        with open(filename, "rb") as f:
            return pickle.load(f)

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
from sklearn import tree
from sklearn.metrics import accuracy_score

class AdaBoostClassifier:
    def __init__(self, n_weakers_limit):
        self.weights = []
        self.classifiers = []
        self.n_weakers_limit = n_weakers_limit
        self.ws = []
        pass

    def is_good_enough(self,X_ver,y_ver):
        if((len(self.classifiers) > self.n_weakers_limit) or
                accuracy_score(y_ver, self.predict(X_ver))>=0.9):
            print(classification_report(y_ver, self.predict(X_ver), digits=4))
            report = f1_score(y_ver, self.predict(X_ver), average=None)
            print('Score: ', report[0] * 0.2 + report[1] * 0.2 + report[2] * 0.6)
            return True
        else:
            print(classification_report(y_ver, self.predict(X_ver), digits=4))
            report = f1_score(y_ver, self.predict(X_ver), average=None)
            print('Score: ', report[0] * 0.2 + report[1] * 0.2 + report[2] * 0.6)
            return False
        pass

    def fit(self,X_train,y_train,X_ver,y_ver):
        w = np.ones((1, len(X_train))) / len(y_train)
        flag = 0
        while (flag == 0 or (not(self.is_good_enough(X_ver,y_ver)))):
        #for i in range(2):
            # 定义并训练基分类器决策树
            flag = 1
            clf = tree.DecisionTreeClassifier\
            (class_weight="balanced", criterion='gini', max_depth=10, splitter="random",random_state=0,min_weight_fraction_leaf=0.0001)
            predit_tree = clf.fit(X_train, y_train, w.reshape(-1))
            #predit_tree = clf.fit(X_train, y_train)
            self.classifiers.append(predit_tree)
            # 计算错误率
            correct_rate = predit_tree.score(X_train, y_train)
            error_rate = 1 - correct_rate
            # 计算分类器权重
            bbb = f1_score(y_ver, predit_tree.predict(X_ver), average=None)
            alpha = bbb[0] * 0.2 + bbb[1] * 0.2 + bbb[2] * 0.6
            #alpha = 1 / 2 * np.log((1 - error_rate) / error_rate)
            # 存储分类器权重
            self.weights.append(alpha)
            # 计算预测结果
            result = predit_tree.predict(X_train).reshape(-1, 1)
            # 更新数据集权重参数
            first_part = -alpha * y_train
            second_part = np.exp(first_part * result)
            third_part = (w.reshape(-1, 1) * second_part)
            z_t = np.sum(third_part)
            w = w.reshape(-1, 1) * second_part / z_t
            self.ws.append(w)
            #print(alpha)
        pass

    def predict(self, X_train, threshold=0):
        scores = []
        lenth = len(self.classifiers)
        mmm_0 = np.zeros((len(X_train),1))
        mmm_1 = np.zeros((len(X_train),1))
        mmm_2 = np.zeros((len(X_train),1))
        
        for i in range(lenth):
            kk = self.classifiers[i].predict(X_train).reshape(-1,1)
            sum_0 = np.where(kk == 0, 1, 0)
            sum_0 = sum_0.astype(np.float64)
            sum_0 *= self.weights[i]
            mmm_0 += sum_0
            #print(mmm_0)
            sum_1 = np.where(kk == 1, 1, 0)
            sum_1 = sum_1.astype(np.float64)
            sum_1 *= self.weights[i]
            mmm_1 += sum_1

            sum_2 = np.where(kk == 2, 1, 0)
            sum_2 = sum_2.astype(np.float64)
            sum_2 *= self.weights[i]
            mmm_2 += sum_2

        mmm = np.concatenate((mmm_0,mmm_1,mmm_2),axis=1)
        return np.argmax(mmm,axis=1)     
        pass


    @staticmethod
    def save(model, filename):
        with open(filename, "wb") as f:
            pickle.dump(model, f)

    @staticmethod
    def load(filename):
        with open(filename, "rb") as f:
            return pickle.load(f)

In [ ]:
alpha = f1_score(y1, AdaBoostClassifiers.classifiers[0].predict(X_train1), average=None)

In [ ]:
alpha

In [ ]:
import numpy as np
a = np.array([[2, 4, 6, 1], [1, 5, 2, 9]])
print(a.shape)
print(np.argmax(a))
print(np.argmax(a, axis=0))  #竖着比较，返回行号
print(np.argmax(a, axis=1))  #横着比较，返回列号

In [ ]:
AdaBoostClassifiers = AdaBoostClassifier(6)

In [ ]:
#AdaBoostClassifiers.fit(X_train,y,X_train1,y1)
AdaBoostClassifiers.fit(X_train1,y1,X_train,y)

In [ ]:
#AdaBoostClassifiers.fit(X_train,y,X_train1,y1)
AdaBoostClassifiers.fit(X_train1,y1,X_train,y)

In [ ]:
AdaBoostClassifiers.classifiers

In [ ]:
train1 = pd.read_csv('is_train_2019070' + str(2) + '.txt')
train1 = train1.merge(attr, on='link', how='left')

In [ ]:
y1 = np.array(train1['label'])
y1 = y1.reshape(-1,1)

In [ ]:
X_train1= train1.loc[:,use_train_feats]
X_train1 = np.array(X_train1)

In [ ]:
print(classification_report(y1, AdaBoostClassifiers.predict(X_train1),target_names=['1','2','3']))

In [ ]:
bbb = f1_score(y1, AdaBoostClassifiers.predict(X_train1), average=None)
print('Score: ', bbb[0] * 0.2 + bbb[1] * 0.2 + bbb[2] * 0.6)

In [ ]:
print(classification_report(y1, AdaBoostClassifiers.classifiers[1].predict(X_train1),target_names=['1','2','3']))

In [ ]:
bbb = f1_score(y1, AdaBoostClassifiers.classifiers[1].predict(X_train1), average=None)
print('Score: ', bbb[0] * 0.2 + bbb[1] * 0.2 + bbb[2] * 0.6)